In [1]:
##Import Required packages
import torch
from torch.autograd import Variable
import torch.nn.functional as F
import torch.utils.data as Data
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
import imageio
import os,sys
import pandas as pd
import re
import seaborn as sns
from tqdm.notebook import tqdm
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler    
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
os.chdir('/content/drive/My Drive/696 Dataset/scratch1/data_netincome/temp')

In [0]:
#Convert embedding files to pickle files
inp_path=r'/content/drive/My Drive/696 Dataset/scratch1/embeddings'
out_path=r'/content/drive/My Drive/696 Dataset/scratch1/data_netincome/emb_pickle/'
error=[]
df=pd.DataFrame(columns=['year','Company','embeddings1a', 'embeddings7'])
df['embeddings1a']=df['embeddings1a'].astype(object)
df['embeddings7']=df['embeddings7'].astype(object)

for filename in os.listdir(inp_path):
    os.chdir(inp_path)
    if filename.endswith(".pt"):   
      try:
        filename1=re.split('[_ .]',filename)
        embedding1=torch.load(filename)['item1a'].cpu().data.numpy()
        embedding2=torch.load(filename)['item7'].cpu().data.numpy()
        
        if os.path.exists(out_path+filename1[0]+'.pkl'):
          os.chdir(out_path)
          df=pd.read_pickle(out_path+filename1[0]+'.pkl')       
          df=df.append(pd.Series([filename1[1], filename1[0], embedding1[0], embedding2[0]], index=df.columns ), ignore_index=True)
          df.to_pickle(filename1[0]+'.pkl')
          df=pd.DataFrame(columns=['year','Company','embeddings1a', 'embeddings7'])
        else:
          df=df.append(pd.Series([filename1[1], filename1[0],embedding1[0], embedding2[0]], index=df.columns ), ignore_index=True)
          df.to_pickle(out_path+filename1[0]+'.pkl')
          df=pd.DataFrame(columns=['year','Company','embeddings1a', 'embeddings7'])
      except:
         error.append(filename)
    
     

In [0]:
import os
import pandas as pd
out_path=r'/content/drive/My Drive/696 Dataset/scratch1/data_netincome/temp/'
os.chdir(out_path)
a=pd.read_pickle('Augmented_Dataset.pkl')

In [0]:
# CONVERT EMBEDDING pickle TO INCLUDE T-1 and T-2 VALUES
out_path=r'/content/drive/My Drive/696 Dataset/scratch1/data_netincome/emb_pickle/'
final_path=r'/content/drive/My Drive/696 Dataset/scratch1/data_netincome/'

for filename in os.listdir(out_path): 
    os.chdir(out_path)
    df=pd.DataFrame()  
    if filename.endswith(".pkl"):  
    
        df=pd.read_pickle(filename) 
        df=df.sort_values(by='year') 
        if len(df.index)>2: 
          #print(filename)  
          #print(df['embeddings1a'])
          temp11=temp17=temp21=temp27=pd.DataFrame()
          
          temp11=df['embeddings1a'].shift(1)
          temp17=df['embeddings7'].shift(1)
          temp21=df['embeddings1a'].shift(2)
          temp27=df['embeddings7'].shift(2)
          df['t-1embedding1a']=temp11
          df['t-1embedding7b']=temp17
          df['t-2embedding1a']=temp21
          df['t-2embedding7b']=temp27
          df.dropna(subset = ["t-2embedding1a",'t-1embedding1a','t-2embedding7b','t-1embedding7b'], inplace=True)
          os.chdir(final_path)
          print(df)
          df.to_pickle(filename)
          
         # print(df['t-2embedding1a'])
        
        

In [0]:
# Add FundamentalAtT value
os.chdir(final_path)
df_struc=df_emb=pd.DataFrame()

for filename in os.listdir(final_path): 
    if filename.endswith('.csv')==True: 
      
      file_temp=filename.split('.')
      
      for filename1 in os.listdir(final_path): 
        if filename1.endswith('.pkl')==True and filename1.endswith('_final.pkl')==False:
          #print(filename,filename1)
          temp=filename1.split('.')
          if temp[0]==file_temp[0]:
            
            df_struc=pd.read_csv(filename)
            df_emb=pd.read_pickle(filename1)
            df_emb['FundamentalAtT']='StructuredDataNotFound'
            for index,row in df_emb.iterrows():
              for i,r in df_struc.iterrows():
                if str(r['year'])==row['year']:
                  
                  df_emb.loc[index,'FundamentalAtT']=r['NetIncomeLoss']
                  print(df_emb)
            df_emb.to_pickle(temp[0]+'_final.pkl')


In [0]:
final_path=r'/content/drive/My Drive/696 Dataset/scratch1/data_netincome/'
os.chdir(final_path)
final=pd.DataFrame()
for filename in os.listdir(final_path): 
  temp=pd.DataFrame()
  if filename.endswith('_final.pkl'):
    
     temp=pd.read_pickle(filename)
     final=pd.concat([final,temp])

final.to_pickle('Augmented_Dataset.pkl')


In [0]:
#Intermediate Dataset:
final_df=pd.DataFrame(columns=['x','y'])
final=pd.read_pickle('Augmented_Dataset.pkl')
#final=final.sort_values(by=index)
for i, r in final.iterrows():
  temp=torch.stack([torch.tensor(r['t-1embedding1a']),torch.tensor(r['t-2embedding1a']),torch.tensor(r['t-1embedding7b']),torch.tensor(r['t-2embedding7b'])],0)
  temp=temp.view(1,-1)
  final_df.loc[i,'x']=temp
  final_df.loc[i,'y']=r['FundamentalAtT']
  #print(final_df)
#final_df.to_pickle('Final_Dataset.pkl')


In [27]:
###### START READING FROM HERE. AUGMENTED FILE PICKED UP FROM GYPSUM



a=pd.read_pickle('Augmented_Dataset.pkl')
print(a.iloc[0])
a = a[a.FundamentalAtT != 'StructuredDataNotFound']
sum1=a['FundamentalAtT'].sum()
#a['FundamentalAtT']=a['FundamentalAtT']/sum1
final_df=pd.DataFrame(columns=['x','y'])

for i, r in a.iterrows():
    temp=np.concatenate([r['t-1embedding1a'],r['t-2embedding1a'],r['t-1embedding7b'],r['t-2embedding7b']],axis=None)
    #print(temp.shape==(3072,))
    #temp=np.concatenate(r['t-1embedding1a']),axis=None)
    #temp=torch.stack([torch.tensor(r['t-1embedding1a']),torch.tensor(r['t-2embedding1a']),torch.tensor(r['t-1embedding7b']),torch.tensor(r['t-2embedding7b'])],0)
    #temp=temp.view(1,-1)
    if temp.shape==(3072,):
      final_df=final_df.append(pd.Series([temp,r['FundamentalAtT']], index=final_df.columns ), ignore_index=True)
      # final_df.loc[i,'x']=temp
      # final_df.loc[i,'y']=r['FundamentalAtT']
  #print(final_df)


print(final_df)

year                                                           2012
Company                                                          PB
embeddings1a      [-0.36747542, -0.3984351, -0.9320981, 0.244671...
embeddings7       [-0.4926114, -0.47682288, -0.93532544, 0.33345...
t-1embedding1a    [-0.32818168, -0.3440001, -0.92424667, 0.21080...
t-1embedding7b    [-0.52081436, -0.4991138, -0.94550407, 0.37271...
t-2embedding1a    [-0.5452079, -0.48004693, -0.97668123, 0.37154...
t-2embedding7b    [-0.5099808, -0.50102955, -0.9516986, 0.344885...
FundamentalAtT                                               167901
Name: 7, dtype: object
                                                      x            y
0     [-0.32818168, -0.3440001, -0.92424667, 0.21080...     167901.0
1     [-0.36747542, -0.3984351, -0.9320981, 0.244671...     221398.0
2     [-0.5640515, -0.4890229, -0.952967, 0.37990147...     297441.0
3     [-0.6171454, -0.5248134, -0.96860147, 0.438356...  286646000.0
4     [-0.54630876, 

In [9]:
final_df.head()

,x,y
0,"[-0.32818168, -0.3440001, -0.92424667, 0.21080...",167901.0
1,"[-0.36747542, -0.3984351, -0.9320981, 0.244671...",221398.0
2,"[-0.5640515, -0.4890229, -0.952967, 0.37990147...",297441.0
3,"[-0.6171454, -0.5248134, -0.96860147, 0.438356...",286646000.0
4,"[-0.54630876, -0.45794713, -0.92361164, 0.3379...",274466000.0


In [0]:
#Train-Test Split
x= final_df['x']
y = final_df['y']
# Train - Test
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1,shuffle=False)
# Split train into train-val
#X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.1, stratify=y_trainval, random_state=21)
y_train, y_test = y_train.astype(float), y_test.astype(float)

# class RegressionDataset(Dataset):   
#     def __init__(self, X_data, y_data):
#         self.X_data = X_data
#         self.y_data = y_data
        
#     def __getitem__(self, index):
#         return self.X_data[index], self.y_data[index]
        
#     def __len__ (self):
#         return len(self.X_data)
# train_dataset = RegressionDataset(torch.from_numpy(X_train).float(), torch.from_numpy(y_train).float())
# val_dataset = RegressionDataset(torch.from_numpy(X_val).float(), torch.from_numpy(y_val).float())
# test_dataset = RegressionDataset(torch.from_numpy(X_test).float(), torch.from_numpy(y_test).float())


In [11]:
print(X_train.iloc[0])
print(y_train[0])

[-0.32818168 -0.3440001  -0.92424667 ... -0.90243673 -0.58014417
  0.14937899]
167901.0


In [0]:
EPOCHS = 200
BATCH_SIZE = 1
LEARNING_RATE = 0.2
NUM_FEATURES = 3072

In [0]:
class Regression(nn.Module):
    def __init__(self, num_features):
        super(Regression, self).__init__()
        
        self.layer_1 = nn.Linear(num_features, 768)
        self.layer_out = nn.Linear(768, 1)
        self.relu = nn.ReLU()

    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.layer_out(x)
        return (x)
    def predict(self, test_inputs):
        x = self.relu(self.layer_1(test_inputs))
        x = self.layer_out(x)
        return (float(x))
        
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model =Regression(NUM_FEATURES)
model.to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)


In [0]:
# train_loader = DataLoader(dataset=X_train, batch_size=BATCH_SIZE)
# #val_loader = DataLoader(dataset=val_dataset, batch_size=1)
# test_loader = DataLoader(dataset=X_test, batch_size=1)

In [18]:
##Train the network
loss_stats = {
    'train': [],
    "val": []
}

print("Begin training.")
for e in tqdm(range(1, EPOCHS+1)):
    
    # TRAINING
    train_epoch_loss = 0
    #model.train()
    for x in X_train:
        # X_train_batch, y_train_batch = X_train_batch.to(device), y_train_batch.to(device)
        optimizer.zero_grad()
        
        y_train_pred = model(torch.tensor(x).unsqueeze(0).cuda())
        #print(y_train_pred,y_train[0])
        
        train_loss = criterion(y_train_pred, torch.tensor(y_train[0]).cuda())
        
        train_loss.backward()
        optimizer.step()
        
        train_epoch_loss += train_loss.item()
        print(train_loss)
        
    # # VALIDATION    
    # with torch.no_grad():
        
    #     val_epoch_loss = 0
        
    #     model.eval()
    #     for X_val_batch, y_val_batch in val_loader:
    #         X_val_batch, y_val_batch = X_val_batch.to(device), y_val_batch.to(device)
            
    #         y_val_pred = model(X_val_batch)
                        
    #         val_loss = criterion(y_val_pred, y_val_batch.unsqueeze(1))
            
    #         val_epoch_loss += val_loss.item()
loss_stats['train'].append(train_epoch_loss/len(X_train.index))
print(f'Epoch {e+0:03}: | Train Loss: {train_epoch_loss/len(X_train.index):.5f} ')
#loss_stats['val'].append(val_epoch_loss/len(val_loader))                              
#print(f'Epoch {e+0:03}: | Train Loss: {train_epoch_loss/len(train_loader):.5f} | Val Loss: {val_epoch_loss/len(val_loader):.5f}')

Begin training.


tensor(2.8191e+10, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(2.3812e+10, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(1.5684e+10, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(5.0841e+09, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(21431258., device='cuda:0', grad_fn=<MseLossBackward>)
tensor(7.0166e+09, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(7.3201e+09, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(2.5802e+09, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(98540672., device='cuda:0', grad_fn=<MseLossBackward>)
tensor(5.7938e+08, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(2.0591e+09, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(3.1232e+09, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(2.9056e+09, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(1.8938e+09, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(7.3861e+08, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(16308094., device='cuda:0', grad_fn=<MseLossBackwa

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:431: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Streaming output truncated to the last 5000 lines.
tensor(294713.2500, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(109426.5703, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(57607.5000, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(4894050., device='cuda:0', grad_fn=<MseLossBackward>)
tensor(272761.3750, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(268291.6250, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(451962.0938, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(215194.5156, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(17510.7324, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(19545380., device='cuda:0', grad_fn=<MseLossBackward>)
tensor(14032516., device='cuda:0', grad_fn=<MseLossBackward>)
tensor(3081999.7500, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(361482.7812, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(7171098., device='cuda:0', grad_fn=<MseLossBackward>)
tensor(11825324., device='cuda:0', grad_fn=<MseLossBackward>)
te

In [25]:
# for i in no_of_rows_in_test:

#   prediction=net(x.float())
#   #Append prediction in file
#   #print(loss_func(prediction,y))
y_pred_list = []
with torch.no_grad():
    model.eval()
    for x in X_test:
        # x = x.to(device)
        y_test_pred = model(torch.tensor(x).cuda())
        #print(y_test_pred)
        y_pred_list.append(y_test_pred.cpu().numpy())
y_pred_list = [a.squeeze().tolist() for a in y_pred_list]
print(y_pred_list,y_test.to_list)
mse = mean_squared_error(y_test, y_pred_list)
r_square = r2_score(y_test, y_pred_list)
print("Mean Squared Error :",mse)
print("R^2 :",r_square)

[168709.703125, 165693.359375, 165040.09375, 164510.546875, 164416.15625, 164194.890625, 163640.84375, 165576.359375, 165007.859375, 165141.359375, 164234.140625, 164094.515625, 162690.890625, 163334.46875, 162266.640625, 162674.21875, 162126.203125, 162179.53125, 163117.609375, 168767.34375, 169181.96875, 169154.625, 169162.46875, 169073.8125, 169232.140625, 170394.3125, 170431.8125, 169381.15625, 168510.421875, 169214.375, 168527.71875, 169182.640625, 169641.171875, 164045.1875, 164251.953125, 165707.765625, 165816.65625, 164407.09375, 164057.21875, 164413.078125, 164691.25, 167278.328125, 166630.015625, 167132.578125, 167332.046875, 166341.296875, 165049.84375, 166050.6875, 167080.796875, 166261.234375, 165929.71875, 165477.390625, 163728.375, 162774.875, 162418.6875, 162688.0, 162781.171875, 162404.984375, 162063.734375, 166178.671875, 164532.890625, 164082.171875, 164424.234375, 164460.34375, 165892.5, 165404.296875, 165671.703125, 160868.515625, 160610.53125, 161097.234375, 16115